In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import psycopg2 # С установкой пакета psycopg2 могут возникнуть проблемы. Лучше установить версию psycopg2-binary
from psycopg2.extras import RealDictCursor
from tqdm.notebook import tqdm

In [ ]:
POSTGRES = {
    'dbname': '<название базы данных>',
    'user': '<имя пользователя>',
    'password': '<пароль>',
    'host': '<имя хоста>',
    'port': 5432 # <номер порта>,
}

In [ ]:
class Postgres:
    def __init__(self):
        try:
            self._conn = psycopg2.connect(**POSTGRES)
        except Exception as conn_error:
            print(conn_error)
            raise conn_error
        self._conn.set_client_encoding('UTF8')

    def execute(self, sql_give, args=None):
        if args is None:
            args = {}

        with self._conn.cursor(cursor_factory=RealDictCursor) as cursor:
            if isinstance(args, list):
                for i in args:
                    cursor.execute(sql_give, i)
            else:
                cursor.execute(sql_give, args)
        self._conn.commit()

    @classmethod
    def fetchall(cls, sql_give):
        pg = cls()
        try:
            return pg._fetchall(sql_give)
        except psycopg2.Error as fetch_error:
            print(fetch_error)
        finally:
            pg.close()

    def _fetchall(self, sql_give):
        with self._conn.cursor(cursor_factory=RealDictCursor) as cursor:
            cursor.execute(sql_give)
            return cursor.fetchall()

    def close(self):
        self._conn.close()

## Простые запросы

### Посмотреть общую схему таблиц в базе данных

In [ ]:
get_schema_sql = """
SELECT table_schema, table_name
FROM information_schema.tables
ORDER BY table_schema,table_name;
"""
conn = Postgres()
data = pd.DataFrame(
    conn.fetchall(get_schema_sql)
)
conn.close()

for table_schema in data.table_schema.unique():
    print(data[data['table_schema'] == table_schema].to_markdown(), end='\n \n')

### Создать таблицу

In [ ]:
create_table_sql = """
CREATE TABLE reports.example_table (
    id integer not null PRIMARY KEY,
    col1 text,
    col2 text,
    col3 text,
    col4 bool
);
CREATE unique index on reports.example_table (id);
"""
# conn = Postgres()
# conn.execute(create_table_sql)
# conn.close()

### Добавить запись в таблицу

In [ ]:
def insert_test(element):
    if isinstance(element, str):
        return element.strip()
    return element

insert_into_sql = """
INSERT INTO reports.example_table (
    id,
    col1,
    col2,
    col3,
    col4
) VALUES (%s,%s,%s,%s,%s);
"""

add = pd.DataFrame([
    [1, '1','A','a', False],
    [2, '2','B','b', True],
    [3, '3','C','c', False]
], columns = ['id','col1','col2','col3','col4'])

# conn = Postgres()

# for row in add.values:
#     display(row)
#     conn.execute(insert_into_sql, 
#                  tuple(insert_test(element) for element in row))
    
# conn.close()

### Получить данные из таблицы

In [ ]:
select_sql = """
SELECT * FROM reports.bot_themes
WHERE meaningful = False AND sub_theme IS NOT NULL;
"""

In [ ]:
select_sql = """
SELECT * FROM reports.example_table
WHERE col4 = False;

"""

# conn = Postgres()
# data = pd.DataFrame(
#     conn.fetchall(select_sql)
# )
# conn.close()

# data

### Изменить таблицу

In [ ]:
alter_table_sql = """
ALTER TABLE reports.example_table
ALTER COLUMN col1 TYPE integer USING col1::integer;
"""

# conn = Postgres()
# conn.execute(alter_table_sql)
# conn.close()

### Удалить записи из таблицы

In [ ]:
delete_values_sql = """
DELETE from reports.common_statistics
WHERE to_char(common_statistics.common__date, 'YYYY-MM-DD HH24:MI:SS') = '2021-05-26 15:37:55';
"""

In [ ]:
delete_values_sql = """
DELETE from reports.example_table
WHERE col4 = True;
"""

# conn = Postgres()
# conn.execute(delete_values_sql)
# conn.close()

### Удалить таблицу

In [ ]:
drop_table_sql = """DROP table reports.example_table;"""

# conn = Postgres()
# conn.execute(drop_table_sql)
# conn.close()